In [6]:
import pandas as pd
%run preprocessing.ipynb
import torch
from torch_geometric.data import Data
import pandas as pd
import networkx as nx


In [7]:
import pandas as pd
from datetime import datetime

# Create a dictionary to map month_id to month and year
month_id_map = {}
for year in range(1990, 2022):
    for month in range(1, 13):
        month_id = (year - 1990) * 12 + month + 120
        month_id_map[month_id] = datetime(year, month, 1)

# Convert month_id to datetime
master_df['date'] = master_df['month_id'].map(month_id_map)


#master_df

,month_id,priogrid_gid,ged_sb,ged_os,ged_ns,ln_pop_gpw_sum,decay_ged_sb_1,decay_ged_sb_25,decay_ged_os_1,decay_ged_sb_5,...,treelag_2_ns,treelag_2_os,sptime_dist_k1_ged_os,sptime_dist_k1_ged_ns,sptime_dist_k10_ged_os,sptime_dist_k10_ged_ns,sptime_dist_k001_ged_os,sptime_dist_k001_ged_ns,month,date
0,121,62356,0.0,0.0,0.0,0.000000,3.125000e-02,3.125000e-02,3.125000e-02,9.765625e-04,...,0.020930,0.018720,17.471405,17.670597,18.384776,17.670597,17.442766,17.240986,1.0,1990-01-01
1,121,79599,0.0,0.0,0.0,8.266445,3.125000e-02,3.125000e-02,3.125000e-02,9.765625e-04,...,0.069700,0.291442,1.414214,11.180340,1.414214,11.704700,0.708237,10.440383,1.0,1990-01-01
2,121,79600,0.0,0.0,0.0,7.805237,3.125000e-02,3.125000e-02,3.125000e-02,9.765625e-04,...,0.074908,0.276007,1.802776,10.735455,1.802776,11.236103,1.118749,9.962510,1.0,1990-01-01
3,121,79601,0.0,0.0,0.0,9.335159,3.125000e-02,3.125000e-02,3.125000e-02,9.765625e-04,...,0.080730,0.180976,2.236068,10.295630,2.236068,10.770330,1.581645,9.486917,1.0,1990-01-01
4,121,80317,0.0,0.0,0.0,12.654427,3.125000e-02,3.125000e-02,3.125000e-02,9.765625e-04,...,0.062183,2.078886,0.500000,11.968709,0.500000,12.500000,0.500000,11.280585,1.0,1990-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5139115,502,190496,0.0,0.0,0.0,10.408626,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,0.069880,0.058797,11.916375,10.594810,14.326549,10.977249,1.524762,10.405138,NaN,2021-10-01
5139116,502,190507,0.0,0.0,0.0,6.647283,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,0.092855,0.059402,8.154753,6.595453,9.899495,6.800735,2.649226,6.519210,NaN,2021-10-01
5139117,502,190508,0.0,0.0,0.0,4.562102,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,0.095266,0.059457,7.889867,6.344289,9.552487,6.519202,2.787185,6.264990,NaN,2021-10-01
5139118,502,190510,0.0,0.0,0.0,7.619576,5.199946e-07,5.199946e-07,5.199946e-07,2.703944e-13,...,0.100458,0.059811,7.433034,5.937171,8.902247,6.041523,3.281524,5.852358,NaN,2021-10-01


In [3]:

# Graph implementation assuming a no neighborhood sampling and no sliding window approach 
def create_graph(master_df):
    graph = nx.Graph()
    unique_priogrid_gids = master_df['priogrid_gid'].unique()

    for priogrid_gid in unique_priogrid_gids:
        node_features = master_df[master_df['priogrid_gid'] == priogrid_gid].drop(['priogrid_gid'], axis=1).to_dict('records')[0]
        graph.add_node(priogrid_gid, **node_features)

    # Add edges connecting each node to its 8 immediate neighbors
    for node in graph.nodes():
        row, col = node // 100, node % 100
        neighbors = []

        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                if dr == 0 and dc == 0:
                    continue
                neighbor_row = row + dr
                neighbor_col = col + dc
                neighbor_priogrid_gid = neighbor_row * 100 + neighbor_col
                if neighbor_priogrid_gid in graph.nodes():
                    neighbors.append(neighbor_priogrid_gid)

        for neighbor in neighbors:
            graph.add_edge(node, neighbor)

    return graph

In [6]:
# Creating a random sliding window of size 8, selecting 4 nodes prior and 4 nodes afte
def sliding_window(master_df):
    G = nx.Graph()

    for gid, group in master_df.groupby('priogrid_gid'):
        features = group[master_df.columns.drop('priogrid_gid')].to_dict('records')[0]
        G.add_node(gid, **features)

    nodes = list(G.nodes())
    for i in range(len(nodes)):
        node = nodes[i]
        neighbors = nodes[max(0, i-4):i] + nodes[i+1:i+5]
        for neighbor in neighbors:
            G.add_edge(node, neighbor)

In [11]:
import torch
import pandas as pd
from torch_geometric.data import Data

def create_graph_pyg(master_df):
    # Prepare node features
    node_features = master_df.drop(columns=['priogrid_gid']).to_numpy(dtype=float)
    node_features_tensor = torch.tensor(node_features, dtype=torch.float)
    
    # Prepare nodes and their indices
    nodes = master_df['priogrid_gid'].tolist()
    node_index = {node: idx for idx, node in enumerate(nodes)}
    
    # Prepare edge list
    edge_list = []
    for i, node in enumerate(nodes):
        # Determine neighbors based on the index; adjust for boundary conditions
        neighbors = nodes[max(0, i-4):i] + nodes[i+1:i+5]
        for neighbor in neighbors:
            if neighbor in node_index:  # Ensure neighbor exists
                edge_list.append([node_index[node], node_index[neighbor]])
    
    # Convert edge list to tensor
    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    
    # Create PyTorch Geometric Data object
    data = Data(x=node_features_tensor, edge_index=edge_index)
    
    return data
# Example usage
# Assume 'master_df' is your DataFrame and has been correctly loaded with 'priogrid_gid' and other features
data = create_graph_pyg(master_df)


In [ ]:
# Implements a neighborhood sampling solution for the graph, considering a randomization factor of 0.50 
import random

def create_graph_neighbouthoodSampling(master_df, sampling_rate=0.5):
    graph = nx.Graph()
    unique_priogrid_gids = master_df['priogrid_gid'].unique()

    for priogrid_gid in unique_priogrid_gids:
        node_features = master_df[master_df['priogrid_gid'] == priogrid_gid].drop(['priogrid_gid'], axis=1).to_dict('records')[0]
        graph.add_node(priogrid_gid, **node_features)
        
        row, col = priogrid_gid // 100, priogrid_gid % 100
        neighbors = []
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                if dr == 0 and dc == 0:
                    continue
                neighbor_row = row + dr
                neighbor_col = col + dc
                neighbor_priogrid_gid = neighbor_row * 100 + neighbor_col
                if neighbor_priogrid_gid in graph.nodes():
                    neighbors.append(neighbor_priogrid_gid)

        for neighbor in neighbors:
            if random.random() <= sampling_rate:
                graph.add_edge(priogrid_gid, neighbor)

    return graph

